In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-gemini
!pip install -q llama-index google-generativeai

  Using cached llama_index_cli-0.4.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_embeddings_openai-0.3.1-py3-none-any.whl.metadata (684 bytes)
  Using cached llama_index_program_openai-0.3.1-py3-none-any.whl.metadata (764 bytes)
  Using cached llama_index_question_gen_openai-0.3.0-py3-none-any.whl.metadata (783 bytes)
  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached striprtf-0.0.26-py3-none-any.whl.metadata (2.1 kB)
  Using cached joblib-1.4.2-py3-none-any.whl

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import Settings
import os

/Users/prajwalbmehendarkar/miniconda3/envs/aix_rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GOOGLE_API_KEY = "Put your Google API key here"
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Setting global parameter
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5") # set the embedding model
Settings.llm = Gemini(model_name="models/gemini-pro")

In [4]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_files=['commands_pdf.pdf']).load_data()

In [5]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=10)

from llama_index.core import Settings

Settings.text_splitter = text_splitter

In [6]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, transformations=[text_splitter])

In [8]:
index.storage_context.persist(persist_dir="blogs/")

In [4]:
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
# storage_context = StorageContext.from_defaults(persist_dir="blogs/")

storage_context = StorageContext.from_defaults(persist_dir="/Users/prajwalbmehendarkar/Documents/rag_blogs/blogs")

# load index
index = load_index_from_storage(storage_context)

In [5]:
template = """
You are a knowledgeable and precise assistant specialized in question-answering tasks, 
particularly from academic and research-based sources. 
Your goal is to provide accurate, concise, and contextually relevant answers based on the given information.

Instructions:

Comprehension and Accuracy: Carefully read and comprehend the provided context from the research paper to ensure accuracy in your response.
Conciseness: Deliver the answer in no more than three sentences, ensuring it is concise and directly addresses the question.
Truthfulness: If the context does not provide enough information to answer the question, clearly state, "I don't know."
Contextual Relevance: Ensure your answer is well-supported by the retrieved context and does not include any information beyond what is provided.

Remember if no context is provided please say you don't know the answer
Here is the question and context for you to work with:

\nQuestion: {question} \nContext: {context} \nAnswer:"""


from llama_index.core.prompts import PromptTemplate

prompt_tmpl = PromptTemplate(
    template=template,
    template_var_mappings={"query_str": "question", "context_str": "context"},
)

In [6]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine



# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template":prompt_tmpl}
)

In [7]:
response = query_engine.query(" Whats the command To set the access control information for the status file with information stored in the acldefs file ")
print(response)

To set the access control information for the status file with information stored in the acldefs file, enter:

aclput -i acldefs status


In [13]:
response = query_engine.query(" Whats the command  To set the access control information for the status file with the same information used for the plans file, enter ")
print(response)

To set the access control information for the status file with the same information used for the plans file, enter:

aclget plans | aclput status


In [14]:
response = query_engine.query(" Whats the command To start the test.c file with a release number of 3.1, use the -r flag with the admin command")
print(response)

To start the test.c file with a release number of 3.1, use the -r flag with the admin command, as shown below:

$ admin -itest.c -r3 s.test.c


In [15]:
response = query_engine.query(" Whats the command To write all of the high-level security options to an output file")
print(response)

The command to write all of the high-level security options to an output file is:
aixpert -l h -n -o /etc/security/aixpert/core/hls.xml


In [17]:
response = query_engine.query(" Whats the command To install a mksysb image on hdisk3 and hdisk4 , then run a customized script (/tmp/script) to copy some user files over to the alternate rootvg file systems before reboot")
print(response)

To install a mksysb image on hdisk3 and hdisk4, then run a customized script (/tmp/script) to copy some user files over to the alternate rootvg file systems before reboot:

alt_disk_mksysb -m /mksysb_images/my_mksysb -d "hdisk3 hdisk4" -s /tmp/script


In [18]:
response = query_engine.query(" Whats the command To install a mksysb image on hdisk1, and to convert the file system from a JFS file system to a JFS2 file system")
print(response)

To install a mksysb image on hdisk1, and to convert the file system from a JFS file system to a JFS2 file system, run the following command:

alt_disk_mksysb -B -T -m  /mksysb_images/my_mksysb -d hdisk1


In [19]:
response = query_engine.query("Who are you")
print(response)

I am a knowledgeable and precise assistant specialized in question-answering tasks, particularly from academic and research-based sources. My goal is to provide accurate, concise, and contextually relevant answers based on the given information.
